## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
solvency_customers = pd.read_csv('/datasets/data.csv')
print(solvency_customers.head(20))
print()
print()
print(solvency_customers.tail(20))
print()
print()
solvency_customers.info()

solvency_customers = solvency_customers.rename(columns={'dob_years':'age'})

solvency_customers.describe()



    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

,children,days_employed,age,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [2]:
solvency_customers[(solvency_customers['total_income'].isnull() == True) & (solvency_customers['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
age                 2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


В строках, где отсутствуют данные в столбце 'days_employed', отсутствуют данные и по 'total_income'.

In [3]:
solvency_customers[(solvency_customers['total_income'].isnull() == True) & (solvency_customers['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Взять среднее по одной группе профессий ('total_income') уже нельзя, будем заполнять NaN по среднему из каждой группы.

Выведем некоторые данные по столбцу 'days_employed' 

In [4]:
print("Количество строк с 'days_employed' < 0:", solvency_customers[solvency_customers['days_employed'] < 0].shape[0])
data_pens = solvency_customers[(solvency_customers['days_employed'] > 0) & (solvency_customers['income_type'] == 'пенсионер')]
print("Количество пенсионеров с 'days_employed' > 0:", data_pens.shape[0])
print("Количество пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20:", data_pens[data_pens['days_employed'] > 33000].shape[0])
print("Среднее количество отработанных дней среди пенсионеров:", data_pens['days_employed'].mean())


Количество строк с 'days_employed' < 0: 15906
Количество пенсионеров с 'days_employed' > 0: 3443
Количество пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20: 3443
Среднее количество отработанных дней среди пенсионеров: 365003.4912448612


In [5]:
print("Количество людей с нулевым возрастом:", solvency_customers[solvency_customers['age'] == 0].count()[0])
print("Количество людей с -1 ребенком:", solvency_customers[solvency_customers['children'] == -1].count()[0])
print("Количество строк с 20 детьми:", solvency_customers[solvency_customers['children'] == 20].count()[0])
print("Количество уникальных людей с 20 детьми:", len(solvency_customers[solvency_customers['children'] == 20]['total_income'].unique()))

Количество людей с нулевым возрастом: 101
Количество людей с -1 ребенком: 47
Количество строк с 20 детьми: 76
Количество уникальных людей с 20 детьми: 68


### Вывод

Возможные ошибки и нестыковки:
I) Минимальное значение по 'children': -1 отрицательное количество детей явная ошибка.
II) Максимальное значение по 'children': 20 вполне вероятно, но стоит проверить на ошибку.
III) Отрицательные значения отработанных дней 'days_employed': необходимо проверить значения и принять по значения по модулю.
IV) Среднее значение по количеству отработанных дней 'days_employed': 63046 дней слишком большое значение, необходимо проверить значения на выбросы.
V) Возраст по некоторым строкам 'age' равен 0 : предполагаю, что просто не указан

I)70% значений из 'days_employed' отрицательные -  возьмем весь столбец по модулю.
II) остальные практически все значения > 0 (кроме двух) - это пенсионеры, причем по всем из них стаж работы > хоть немного разумных 90 лет.
III) нулевой возраст для некоторых клиентов - надо исправлять.
IV) "-1 ребенок" - надо исправлять.
V) "20 детей" - надо исправлять.

### Шаг 2. Предобработка данных

### Обработка пропусков

Для каждого типа 'total_income' выведем следующее:

In [6]:
data_grouped = solvency_customers.groupby('total_income').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

dict_to_rename = dict(zip(data_grouped.columns.levels[1], ['Общее количество строк', 'Среднее', 'Кол-во значений > 0']))

data_grouped = data_grouped.rename(columns=dict_to_rename, level=1)
data_grouped

days_employed                                   
             Общее количество строк        Среднее Кол-во значений > 0
total_income                                                          
2.066726e+04                      1  359219.059341                 1.0
2.120528e+04                      1  369708.589113                 1.0
2.136765e+04                      1   -3642.820023                 0.0
2.169510e+04                      1  359726.104207                 1.0
2.189561e+04                      1  346602.453782                 1.0
...                             ...            ...                 ...
1.711309e+06                      1   -5734.127087                 0.0
1.715018e+06                      1   -4719.273476                 0.0
1.726276e+06                      1   -5248.554336                 0.0
2.200852e+06                      1   -2577.664662                 0.0
2.265604e+06                      1   -1477.438114                 0.0

[19351 rows x 3 columns]

по отработанным дням ('days_employed') однотипны:  все значения > 0, по остальным - < 0, поэтому можем безболезненно применять abs(), считать среднее.

In [7]:
solvency_customers[['total_income', 'days_employed']] = solvency_customers[['total_income', 'days_employed']].abs()
solvency_customers['ratio_days_employed'] = solvency_customers[solvency_customers['days_employed'].notnull()]['days_employed']/((solvency_customers['age']-16)*365)
solvency_customers.head()
solvency_customers = solvency_customers.drop('ratio_days_employed', axis=1)
solvency_customers.info()
solvency_customers['days_employed'] = solvency_customers.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*solvency_customers['age']*365))
solvency_customers['total_income'] = solvency_customers.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
solvency_customers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [8]:
# 'education' в low-индекс
solvency_customers['education'] = solvency_customers['education'].str.lower()
# 'children' (-1) на 0
print("Количество строк с 'children' -1 -", solvency_customers[solvency_customers['children'] == -1].count()[0])
print("Количество строк с 'children' 0 -", solvency_customers[solvency_customers['children'] == 0].count()[0])
print("Количество строк с 'children' 20 -", solvency_customers[solvency_customers['children'] == 20].count()[0])

Количество строк с 'children' -1 - 47
Количество строк с 'children' 0 - 14149
Количество строк с 'children' 20 - 76


In [9]:
solvency_customers['children'] = solvency_customers['children'].replace(-1, 0)
print("Количество строк с 'dob_years' 0 -", solvency_customers[solvency_customers['age'] == 0].count()[0])
print("Количество строк с 'dob_years' < 19 -", solvency_customers[solvency_customers['age'] < 19].count()[0])

Количество строк с 'dob_years' 0 - 101
Количество строк с 'dob_years' < 19 - 101


количество нулей по возрасту (столбец 'dob_years') и количество несовершеннолетних

In [10]:
solvency_customers['days_employed'] = solvency_customers['days_employed'].astype('int')
solvency_customers['total_income'] = solvency_customers['total_income'].astype('int')
solvency_customers.dtypes

children             int64
days_employed        int64
age                  int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

Месячную зарплату и стаж в днях перевели в 'int', чтобы видеть целочисленные значения.

In [11]:
solvency_customers = solvency_customers.drop_duplicates()
solvency_customers.duplicated().sum()

0

### Вывод

Очистка датафрейма и дубликатов произведена.

### Лемматизация

выведем список уникальных целей ('purpose')

In [12]:
unique_purposes = solvency_customers['purpose'].value_counts().index.tolist()
print(solvency_customers['purpose'].value_counts().to_frame())

                                        purpose
свадьба                                     791
на проведение свадьбы                       768
сыграть свадьбу                             765
операции с недвижимостью                    675
покупка коммерческой недвижимости           661
операции с жильем                           652
покупка жилья для сдачи                     651
операции с коммерческой недвижимостью       650
жилье                                       646
покупка жилья                               646
покупка жилья для семьи                     638
строительство собственной недвижимости      635
недвижимость                                633
операции со своей недвижимостью             627
строительство жилой недвижимости            624
покупка недвижимости                        621
покупка своего жилья                        620
строительство недвижимости                  619
ремонт жилью                                607
покупка жилой недвижимости              

Загрузим pymystem3 и лемматизируем полученный выше список, сверстаем "рейтинг"

In [13]:
from pymystem3 import Mystem
from collections import Counter

In [14]:
m = Mystem()
list_of_lemmas = []
for element in solvency_customers['purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Возьмем топ-8 без пробелов и слова длиной 4 и более букв

In [15]:
final_list_of_purposes = [k for k in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(k) > 4 if k != ' ' if k != '\n'][0:8]
print(final_list_of_purposes)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство']


In [16]:
final_list_of_purposes.remove('покупка') 
final_list_of_purposes.remove('строительство')
final_list_of_purposes.remove('операция')
print(final_list_of_purposes)

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']


In [17]:
from nltk.stem import SnowballStemmer 

In [18]:
russian_stemmer = SnowballStemmer('russian')

stemmed_purposes = [russian_stemmer.stem(word) for word in final_list_of_purposes]
print(stemmed_purposes)

['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']


In [19]:
dict_stemmed_purposes = {v:k for k,v in enumerate(stemmed_purposes)}
dict_stemmed_purposes['недвижим'] = 1
print(dict_stemmed_purposes)

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}


### Вывод

Список целей получения кредита в виде словаря {цель:номер типа}.
Далее, создадим новый столбец и дадим каждой строке категорию из этого списка и проверим.

### Категоризация данных

In [20]:
solvency_customers['purpose_cat'] = 0
for row in range(len(solvency_customers)):
    for purpose in dict_stemmed_purposes:
        if purpose in solvency_customers.iloc[row, 11]:
            solvency_customers.iloc[row, 12] = dict_stemmed_purposes[purpose]
        
solvency_customers.head(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


In [21]:
solvency_customers[solvency_customers['purpose_cat'] == 0].count().sum()

0

In [22]:
solvency_customers.groupby(['education', 'education_id']).size().to_frame('count').reset_index()

,education,education_id,count
0,высшее,0,5250
1,начальное,3,282
2,неоконченное высшее,2,744
3,среднее,1,15172
4,ученая степень,4,6


In [23]:
solvency_customers.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index()

,family_status,family_status_id,count
0,Не женат / не замужем,4,2810
1,в разводе,3,1195
2,вдовец / вдова,2,959
3,гражданский брак,1,4151
4,женат / замужем,0,12339


In [24]:
def children_cat(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
solvency_customers['children_cat'] = solvency_customers.apply(children_cat, axis=1)
def salary_cat(row):
    if row['total_income'] <= 50000:
        return 'бедный'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] < 1000000:
        return 'зажиточный'
    else:
        return 'миллионер'
solvency_customers['salary_cat'] = solvency_customers.apply(salary_cat, axis=1)
solvency_customers.head()


,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,children_cat,salary_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,1-2 ребенка,зажиточный
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,1-2 ребенка,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,нет детей,зажиточный
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,многодетные,зажиточный
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,нет детей,зажиточный


### Вывод

Категоризировали датафрейм по следующим данным:
-'education' и 'family_status_id'

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
def relation(category):
    return solvency_customers.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')
relation('children_cat')

,debt
children_cat,
нет детей,0.075258
многодетные,0.085526
1-2 ребенка,0.093003


### Вывод

Взявщие кредит, не имеющие детей, менее склонны к просрочке по выплатам кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
relation('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


### Вывод

Взявщие кредит, не имеют узаконенных отношений, более склонны к просрочке по выплатам кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
relation('salary_cat')
solvency_customers.describe()

,children,days_employed,age,education_id,family_status_id,debt,total_income,purpose_cat
count,21454.000000,2.145400e+04,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04,21454.000000
mean,0.542137,1.424659e+08,43.271231,0.817097,0.973898,0.081150,1.674316e+05,1.899786
std,1.381794,1.041747e+09,12.570822,0.548674,1.421567,0.273072,9.806060e+04,1.055592
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.066700e+04,1.000000
25%,0.000000,1.020000e+03,33.000000,1.000000,0.000000,0.000000,1.076230e+05,1.000000
50%,0.000000,2.588000e+03,42.000000,1.000000,0.000000,0.000000,1.518870e+05,1.000000
75%,1.000000,3.325852e+05,53.000000,1.000000,1.000000,0.000000,2.024170e+05,3.000000
max,20.000000,9.725518e+09,75.000000,4.000000,4.000000,1.000000,2.265604e+06,4.000000


### Вывод

Взявщие кредит, с уровнем дохода < 50000р. менее склонны нарушать обязательства по выплатам кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
relation('purpose_cat')

,debt
purpose_cat,
1,0.072334
4,0.080034
3,0.092200
2,0.093590


### Вывод

Взявщие кредит,  берущие кредит для приобретения/проведение операций с жильем, наиболее ответственны и менее склонны нарушать обязательства по выплатам кредита в срок.

### Шаг 4. Общий вывод

Семеное положение и количество детей влияет на факт погашения кредита в строк:
-заемщики с официально оформленными отношениями и не имеющие детей - самые ответственные заемщики;
-заемщики, состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей - самые менее ответственные заемщики.
В целом:
Ответственный заещик:
Не имеет детей	
Находится/был в официальном браке	
Уровень дохода < 50000	
Берет кредит на жилье
Менее ответственный заещик:
Имеет 1-2 детей	
Не в узаконенных отношениях/не в отношениях	
Уровень дохода > 50000	
Берет кредит на образование/свадьбу
